In [1]:
import sys
sys.path.append("/home/fehrdelt/data_ssd/MedicalImaging_GIN/gradient_boosting")

import os
import lightgbm as lgb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


from sklearn.impute import SimpleImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.manifold import TSNE
import umap

import matplotlib.pyplot as plt




/data_local/data_ssd/fehrdelt/environments/general_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_DIRECTORY = "/home/fehrdelt/data_ssd/data/clinical_data/"

In [3]:
X_volumes_clinical = pd.read_csv(DATA_DIRECTORY+"combined_clinical_data_volumes_outcome_TTS_LDDMM.csv", usecols=range(2,31))
X_volumes_clinical.head()

,supratentorial_IPH,supratentorial_SAH,supratentorial_Petechiae,supratentorial_Edema,infratentorial_IPH,infratentorial_SAH,infratentorial_Petechiae,infratentorial_Edema,brainstem_IPH,brainstem_SAH,...,pression_arterielle_diastolique_PAD_arrivee_du_smur,score_glasgow_initial,score_glasgow_moteur_initial,anomalie_pupillaire_prehospitalier,frequence_cardiaque_FC_arrivee_du_smur,arret_cardio_respiratoire_massage,penetrant_objet,ischemie_du_membre,hemorragie_externe,amputation
0,0,337,0,0,0,15,0,0,0,0,...,49.0,15.0,6.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,60.0,15.0,6.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
2,0,90,0,0,0,0,0,0,0,0,...,64.0,14.0,6.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0
3,0,333,0,0,0,0,0,0,0,0,...,71.0,15.0,6.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0
4,0,17,0,15,0,0,0,0,0,0,...,79.0,NaN,NaN,0.0,83.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X_clinical_only = pd.read_csv(DATA_DIRECTORY+"combined_clinical_data_volumes_outcome_TTS_LDDMM.csv", usecols=range(16,31))
X_clinical_only.head()

,age,hemocue_initial,fracas_du_bassin,catecholamines,pression_arterielle_systolique_PAS_arrivee_du_smur,pression_arterielle_diastolique_PAD_arrivee_du_smur,score_glasgow_initial,score_glasgow_moteur_initial,anomalie_pupillaire_prehospitalier,frequence_cardiaque_FC_arrivee_du_smur,arret_cardio_respiratoire_massage,penetrant_objet,ischemie_du_membre,hemorragie_externe,amputation
0,52.0,NaN,0.0,0.0,87.0,49.0,15.0,6.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0
1,23.0,NaN,0.0,0.0,100.0,60.0,15.0,6.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
2,42.0,13.1,0.0,0.0,101.0,64.0,14.0,6.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0
3,34.0,15.8,0.0,0.0,110.0,71.0,15.0,6.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0
4,22.0,NaN,0.0,0.0,114.0,79.0,NaN,NaN,0.0,83.0,0.0,0.0,0.0,0.0,0.0


In [5]:
y = pd.read_csv(DATA_DIRECTORY+"combined_clinical_data_volumes_outcome_TTS_LDDMM.csv", usecols=[31])
y.head()

,outcome_neurochir_pic
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [6]:
nan_indexes = y.loc[pd.isna(y["outcome_neurochir_pic"]), :].index # indexes where there is a nan value.
print(nan_indexes)

y = y.dropna()
X_volumes_clinical = X_volumes_clinical.drop(nan_indexes)
X_clinical_only = X_clinical_only.drop(nan_indexes)

Index([59, 80, 106, 108, 117, 123, 130], dtype='int64')


In [7]:
imp = SimpleImputer(missing_values=np.nan, strategy="median")

imp.fit(X_volumes_clinical)
X_volumes_clinical = imp.transform(X_volumes_clinical)

imp.fit(X_clinical_only)
X_clinical_only = imp.transform(X_clinical_only)

Take test set apart
test set: 10% -> first 32 patients with 2 neurochir patients

In [8]:
list_outcome = list(y["outcome_neurochir_pic"])

print(f"Total nb of patients: {len(list_outcome)}")
print(f"Total nb of neurochir+pic: {np.sum(list_outcome)}")
print(f"% of neurochir: {100*np.sum(list_outcome)/len(list_outcome)}%")

Total nb of patients: 321
Total nb of neurochir+pic: 19.0
% of neurochir: 5.919003115264798%


In [9]:
print(len(y))
print(len(X_volumes_clinical))

321
321


In [10]:
y = y['outcome_neurochir_pic'].to_numpy()
y = [int(i) for i in y]

y_test = y[:32]
y_dev = y[32:]

X_volumes_clinical_test = X_volumes_clinical[:32]
X_volumes_clinical_dev = X_volumes_clinical[32:]

X_clinical_only_test = X_clinical_only[:32]
X_clinical_only_dev = X_clinical_only[32:]

Training

Clinical data only

In [11]:
print(len(y_dev))
print(len(X_clinical_only_dev))

print("---")
print(len(y_test))
print(len(X_volumes_clinical_test))

289
289
---
32
32


In [12]:
#model = DecisionTreeClassifier()
model = HistGradientBoostingClassifier(categorical_features=[False, False, True, True, False, False, False, False, True, False, True, True, True, True, True])

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

#scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
scores = cross_validate(model, X_clinical_only_dev, y_dev, scoring=['roc_auc', 'recall_macro'], cv=cv, n_jobs=-1)

#print(scores)

print('Mean ROC AUC: %.3f' % np.mean(scores['test_roc_auc']))
print('Mean recall macro: %.3f' % np.mean(scores['test_recall_macro']))

Mean ROC AUC: 0.821
Mean recall macro: 0.603


Segmentation volumes and clinical data

In [13]:
#model = DecisionTreeClassifier()
model = HistGradientBoostingClassifier(categorical_features=[False]*14 + [False, False, True, True, False, False, False, False, True, False, True, True, True, True, True])

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)

#scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
scores = cross_validate(model, X_volumes_clinical_dev, y_dev, scoring=['roc_auc', 'recall_macro'], cv=cv, n_jobs=-1)

#print(scores)

print('Mean ROC AUC: %.3f' % np.mean(scores['test_roc_auc']))
print('Mean recall macro: %.3f' % np.mean(scores['test_recall_macro']))

Mean ROC AUC: 0.934
Mean recall macro: 0.625


Test inference

In [14]:
categorical = [False]*14 + [False, False, True, True, False, False, False, False, True, False, True, True, True, True, True]
clf_clinical_volumes = HistGradientBoostingClassifier(categorical_features=categorical).fit(X_volumes_clinical_dev, y_dev)
y_pred = clf_clinical_volumes.predict(X_volumes_clinical_test)
print(y_pred)
print(classification_report(y_pred=y_pred, y_true=y_test))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.94      0.97      0.95        30
           1       0.00      0.00      0.00         2

    accuracy                           0.91        32
   macro avg       0.47      0.48      0.48        32
weighted avg       0.88      0.91      0.89        32



In [15]:
categorical = [False, False, True, True, False, False, False, False, True, False, True, True, True, True, True]
clf_clinical_only = HistGradientBoostingClassifier(categorical_features=categorical).fit(X_clinical_only_dev, y_dev)
y_pred = clf_clinical_only.predict(X_clinical_only_test)
print(y_pred)
print(classification_report(y_pred=y_pred, y_true=y_test))

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        30
           1       1.00      0.50      0.67         2

    accuracy                           0.97        32
   macro avg       0.98      0.75      0.83        32
weighted avg       0.97      0.97      0.96        32

